# Processing

In [8]:
import numpy as np
import pandas as pd
import os

from classes_ico import Vehicle, Customer, VRPTW
import geopy

#### Directories

In [2]:
ROOT_DIR = os.path.abspath('..')
DATA_DIR = os.path.join(ROOT_DIR, 'data')

CUSTOMER_DIR = os.path.join(DATA_DIR, '2_detail_table_customers.xls')
VEHICLES_DIR = os.path.join(DATA_DIR, '3_detail_table_vehicles.xls')
DEPOTS_DIR = os.path.join(DATA_DIR, '4_detail_table_depots.xls')
CONSTRAINTS_DIR = os.path.join(DATA_DIR, '5_detail_table_constraints_sdvrp.xls')
DEPOTS_DISTANCES_DIR = os.path.join(DATA_DIR, '6_detail_table_cust_depots_distances.xls')
CUSTOMER_DISTANCES_DIR = os.path.join(DATA_DIR, '7_detail_table_cust_cust_distances.xls')

In [3]:
customers = pd.read_excel(CUSTOMER_DIR)
vehicles = pd.read_excel(VEHICLES_DIR)
depots = pd.read_excel(DEPOTS_DIR)
constraints = pd.read_excel(CONSTRAINTS_DIR)
depots_dist = pd.read_excel(DEPOTS_DISTANCES_DIR)
customers_dist = pd.read_excel(CUSTOMER_DISTANCES_DIR)

#### Fonctions load

In [14]:
def load_vehicle(vehicles, MODE="mean", vehicle_nb=None):
    if vehicle_nb:
        volume = vehicles[vehicles["VEHICLE_CODE"]==vehicle_nb]["VEHICLE_TOTAL_VOLUME_M3"]
        weight = vehicles[vehicles["VEHICLE_CODE"]==vehicle_nb]["VEHICLE_TOTAL_WEIGHT_KG"]
        cost_km = vehicles[vehicles["VEHICLE_CODE"]==vehicle_nb]["VEHICLE_VARIABLE_COST_KM"]  
    else :
        volume = getattr(vehicles["VEHICLE_TOTAL_VOLUME_M3"], MODE)()
        weight = getattr(vehicles["VEHICLE_TOTAL_WEIGHT_KG"], MODE)()
        cost_km = getattr(vehicles["VEHICLE_VARIABLE_COST_KM"], MODE)()  
    vehicle = Vehicle(volume, weight, cost_km)
    return vehicle

In [24]:
def load_customers(CUSTOMER_DIR):
    customers = pd.read_excel(CUSTOMER_DIR)
    customers = customers.drop_duplicates(subset=["CUSTOMER_CODE"], keep='first')
    id = 0
    latitude = depots.loc[0,"DEPOT_LATITUDE"]
    longitude = depots.loc[0,"DEPOT_LONGITUDE"]
    time_window = (depots.loc[0,"DEPOT_AVAILABLE_TIME_FROM_MIN"], depots.loc[0,"DEPOT_AVAILABLE_TIME_TO_MIN"])
    request_volume =0
    request_weight = 0
    time_service = 0
    depot = Customer(id, latitude, longitude, time_window, request_volume, request_weight, time_service)
    list_customers = [depot]
    for i, code in enumerate(customers["CUSTOMER_CODE"]):
        id = i
        latitude = customers[customers["CUSTOMER_CODE"]==code]["CUSTOMER_LATITUDE"]
        longitude = customers[customers["CUSTOMER_CODE"]==code]["CUSTOMER_LONGITUDE"]
        time_window = (customers[customers["CUSTOMER_CODE"]==code]["CUSTOMER_TIME_WINDOW_FROM_MIN"], 
                       customers[customers["CUSTOMER_CODE"]==code]["CUSTOMER_TIME_WINDOW_TO_MIN"])
        request_volume = customers[customers["CUSTOMER_CODE"]==code]["TOTAL_VOLUME_M3"]
        request_weight = customers[customers["CUSTOMER_CODE"]==code]["TOTAL_WEIGHT_KG"]
        time_service = customers[customers["CUSTOMER_CODE"]==code]["CUSTOMER_DELIVERY_SERVICE_TIME_MIN"]
        customer = Customer(id, latitude, longitude, time_window, request_volume, request_weight, time_service)
        list_customers.append(customer)
    return list_customers

In [25]:
print(len(load_customers(customers)))

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [21]:
customers["CUSTOMER_CODE"].value_counts()

1805      8
7634      7
921003    7
1408      6
10825     6
         ..
1243      1
884       1
136404    1
136403    1
136150    1
Name: CUSTOMER_CODE, Length: 573, dtype: int64